# Tutorial: Product Unit Model with Modular Property Package

In [ ]:
from IPython.core.display import SVG
SVG(filename='product.svg')

## Learning Outcomes

- Demonstrate use of the Product unit model in IDAES


## Problem Statement

In this tutorial, we will be feeding a mixed stream of liquid benzene and liquid toluene. This unit model has no outlets and only one inlet. The inlet conditions are as follows:

Flow Rate = 100 mol/s

Mole Fraction (benzene) = 0.6

Mole Fraction (toluene) = 0.4

Temperature = 298 K

Pressure = 101325 Pa

For more details, please refer to the IDAES documentation: https://idaes-pse.readthedocs.io/en/stable

## Setting up the problem in IDAES

In the following cell, we will be importing the necessary components from Pyomo and IDAES.

In [ ]:
# Import objects from pyomo package 
from pyomo.environ import ConcreteModel, value

# Import the solver
from idaes.core.solvers import get_solver

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import the product unit model
from idaes.models.unit_models import Product

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the modular property package to create a property block for the flowsheet
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock

# Import the BT_Ideal property package to create a configuration file for the GenericParameterBlock
from idaes.models.properties.modular_properties.examples.BT_ideal import configuration

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Create the ConcreteModel and the FlowsheetBlock objects, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False) # dynamic or ss flowsheet needs to be specified here

# Add properties parameter block to the flowsheet with the appropriate configuration file
m.fs.properties = GenericParameterBlock(**configuration)

In the following cell, we will be creating the product unit model, assigning a property package to it, and determining the degrees of freedom associated with the product unit model.

In [ ]:
# Create an instance of the product unit, attaching it to the flowsheet
# Specify that the property package to be used with the product is the one we created earlier

m.fs.product = Product(property_package=m.fs.properties)

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print('The initial degrees of freedom are: {0}'.format(DOF_initial))

In [ ]:
assert DOF_initial == 5

In the following cell, we will be specifying the inlet conditions for the product block and ensuring that there are zero degrees of freedom.

In [ ]:
# Fix the inlet conditions

# Product stream
m.fs.product.flow_mol.fix(100) # converting to mol/s as unit basis is mol/s
m.fs.product.mole_frac_comp[0, "benzene"].fix(0.6)
m.fs.product.mole_frac_comp[0, "toluene"].fix(0.4)
m.fs.product.pressure.fix(101325) # Pa
m.fs.product.temperature.fix(298) # K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print('The final degrees of freedom is: {0}'.format(DOF_final))

In [ ]:
assert DOF_final == 0

### Flowsheet Initialization

In [ ]:
# Initialize the flowsheet, and set the output at WARNING
m.fs.product.initialize(outlvl=idaeslog.WARNING)

### Obtaining Simulation Results

In [ ]:
# Solve the simulation using the IDAES solver
# Note: If the degrees of freedom = 0, we have a square problem
solver = get_solver()
solver.solve(m, tee=True)

In [ ]:
from pyomo.opt import TerminationCondition, SolverStatus

# Check if termination condition is optimal
assert result.solver.termination_condition == TerminationCondition.optimal
assert result.solver.status == SolverStatus.ok

### View Results

As expected, the product shown below matches the feed conditions that were set earlier.

In [ ]:
# Display output report
m.fs.product.report()

In [ ]:
import pytest

# Check results
assert value(m.fs.product.inlet.flow_mol[0]) == pytest.approx(100, rel=1e-6)
assert value(m.fs.product.mole_frac_comp[0, "benzene"]) == pytest.approx(0.6, rel=1e-6)
assert value(m.fs.product.mole_frac_comp[0, "toluene"]) == pytest.approx(0.4, rel=1e-6)